In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

In [2]:
load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

# OpenAI의 임베딩 모델(text-embedding-ada-002)을 초기화.
embeddings = OpenAIEmbeddings(
    openai_api_key=key,
    model='text-embedding-ada-002'
)

# Chroma 벡터 스토어를 초기화하고, 임베딩 데이터를 ./data 디렉토리에 저장.
database = Chroma(
    persist_directory = './data', 
    embedding_function = embeddings
)

def ask(query):
    documents = database.similarity_search(query)	# 질문과 유사한 문서를 검색

    document_string = ''    

    for document in documents:						# 검색된 문서들을 document_string에 저장
        document_string += f"""
	
-------------------------------						
{document.page_content}
	"""
    template="""문장을 바탕으로 질문에 답하세요.
    문장: {document}
    질문: {query}
    """

    prompt = PromptTemplate(
        template=template, 
        input_variables=['document', 'query']
    )

    llm = ChatOpenAI(
        api_key=key, 
        model='gpt-4o',
        temperature=0
    )

    
    chain = prompt | llm

    return chain.invoke({'document': document_string, 'query': query})

d:\kodefare_sample\gpt_speak_lib_app\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
prompt = '우리 도서관에 파이썬 책이 있나요?'
response = ask(prompt)
msg = response.content

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [4]:
msg

'네, 우리 도서관에는 파이썬 책이 있습니다. 공학 섹션(#500)에 다음과 같은 파이썬 관련 책들이 있습니다:\n\n- 초보자를 위한 파이썬(Python) 200제\n- 파이썬으로 시작하는 데이터 분석\n- 누구나 할 수 있다 파이썬 Python 기초\n- OpenAI API와 파이썬으로 나만의 챗GPT 만들기'